# **Finetuning Bart-Large-CNN Model** 
###Trained on Multi-News Dataset
Adapted from Source: 
https://colab.research.google.com/github/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb

## Installing packages

In [ ]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 56.4 MB/s 
     |████████████████████████████████| 163 kB 67.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.8 MB/s 
     |████████████████████████████████| 212 kB 69.3 MB/s 
     |████████████████████████████████| 115 kB 67.7 MB/s 
     |████████████████████████████████| 127 kB 62.1 MB/s 
     |████████████████████████████████| 115 kB 64.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 
Looking in indexes: https

In [ ]:
!pip install git+https://github.com/huggingface/datasets.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-o49d78fg
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-o49d78fg
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for datasets: filename=datasets-2.6.2.dev0-py3-none-any.whl size=441891 sha256=fb805674f3058fd5003d5a8c807ce629650959b676d40627e9113fc4853c5824
  Stored in directory: /tmp/pip-ephem-wheel-cache-ab97zuhd/wheels/cc/33/c1/2cacc415b23189a83908e45db67381ba26175ef1e8aa9062aa
Successfully built datasets
  Attempting uninstall: datasets
    Found existing installation: datasets 2.6.1
    Uninstalling datasets-2.6.1:
      Successfully uninstalled datasets-2.6.1


In [ ]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


## Loading data

In [ ]:
from datasets import load_dataset, load_metric
train_data_txt = load_dataset("multi_news", download_mode="force_redownload", split = "train")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


In [ ]:
val_data_txt = load_dataset("multi_news", download_mode="force_redownload", split = "validation")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


In [ ]:
train_data_txt

Dataset({
    features: ['document', 'summary'],
    num_rows: 44972
})

In [ ]:
val_data_txt

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [ ]:
train_data_txt = train_data_txt.select(range(1000))
validation_data_txt = val_data_txt.select(range(100))

In [ ]:
import pandas as pd

## Data preprocessing

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn", padding="encoder_max_length", truncation="only_first")
encoder_max_length = 1024  
decoder_max_length = 512

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["document"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation="only_first", max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation="only_first", max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Loading model and metrics

In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

In [ ]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """


## Model training, evaluation and deployment

In [ ]:
# Defining training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="bart_large_summarise_v3",
    num_train_epochs=10,  
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=10,
    save_total_limit=3,
    push_to_hub=True
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Pass training argumnets, model, tokenizer, datasets and compute_metrics function to the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/debbiesoon/bart_large_summarise_v3 into local empty directory.


In [ ]:
# Train model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Step,Training Loss
10,3.811600
20,3.816700
30,3.870800
40,3.703800
50,3.691400
60,3.711200
70,3.661500
80,3.509800
90,3.547000
100,3.596400


Saving model checkpoint to bart_large_summarise_v3/checkpoint-500
Configuration saved in bart_large_summarise_v3/checkpoint-500/config.json
Model weights saved in bart_large_summarise_v3/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bart_large_summarise_v3/checkpoint-500/tokenizer_config.json
Special tokens file saved in bart_large_summarise_v3/checkpoint-500/special_tokens_map.json
tokenizer config file saved in bart_large_summarise_v3/tokenizer_config.json
Special tokens file saved in bart_large_summarise_v3/special_tokens_map.json
Saving model checkpoint to bart_large_summarise_v3/checkpoint-1000
Configuration saved in bart_large_summarise_v3/checkpoint-1000/config.json
Model weights saved in bart_large_summarise_v3/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in bart_large_summarise_v3/checkpoint-1000/tokenizer_config.json
Special tokens file saved in bart_large_summarise_v3/checkpoint-1000/special_tokens_map.json


Training completed. Do not forg

TrainOutput(global_step=1250, training_loss=2.7135862167358398, metrics={'train_runtime': 44914.232, 'train_samples_per_second': 0.223, 'train_steps_per_second': 0.028, 'total_flos': 2.167104602112e+16, 'train_loss': 2.7135862167358398, 'epoch': 10.0})

In [ ]:
# Generating evaluation metrics
results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


In [ ]:
print(results)

{'eval_loss': 4.135930061340332, 'eval_rouge1': 0.3914, 'eval_rouge2': 0.1399, 'eval_rougeL': 0.2039, 'eval_rougeLsum': 0.3504, 'eval_gen_len': 141.64, 'eval_runtime': 662.3506, 'eval_samples_per_second': 0.151, 'eval_steps_per_second': 0.02, 'epoch': 10.0}


In [ ]:
# Pushing model onto HuggingFace Hub
trainer.push_to_hub()

Saving model checkpoint to bart_large_summarise_v3
Configuration saved in bart_large_summarise_v3/config.json
Model weights saved in bart_large_summarise_v3/pytorch_model.bin
tokenizer config file saved in bart_large_summarise_v3/tokenizer_config.json
Special tokens file saved in bart_large_summarise_v3/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.51G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/debbiesoon/bart_large_summarise_v3
   d4b7c35..e920a9b  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/debbiesoon/bart_large_summarise_v3
   d4b7c35..e920a9b  main -> main

To https://huggingface.co/debbiesoon/bart_large_summarise_v3
   e920a9b..cb3e675  main -> main

   e920a9b..cb3e675  main -> main



'https://huggingface.co/debbiesoon/bart_large_summarise_v3/commit/e920a9b5d7d9123c854c7b7ebbd45697c90839c9'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save model to Google Drive
trainer.save_model('/content/drive/MyDrive/SGH Project/bart_large_summarise_v3/')

Saving model checkpoint to /content/drive/MyDrive/SGH Project/bart_large_summarise_v3/
Configuration saved in /content/drive/MyDrive/SGH Project/bart_large_summarise_v3/config.json
Model weights saved in /content/drive/MyDrive/SGH Project/bart_large_summarise_v3/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/SGH Project/bart_large_summarise_v3/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/SGH Project/bart_large_summarise_v3/special_tokens_map.json
Saving model checkpoint to bart_large_summarise_v3
Configuration saved in bart_large_summarise_v3/config.json
Model weights saved in bart_large_summarise_v3/pytorch_model.bin
tokenizer config file saved in bart_large_summarise_v3/tokenizer_config.json
Special tokens file saved in bart_large_summarise_v3/special_tokens_map.json
